In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

while 'notebooks' in os.getcwd():
    os.chdir('..')

import pandas as pd
import torch
import torch.nn.functional as F
from ogb.nodeproppred import DglNodePropPredDataset, Evaluator
from sklearn.metrics import roc_auc_score
import logging
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.offline as pyo
import numpy as np

from src.dgl_models import GraphSAGE, EdgeWeigher
from src.data.node_classifier.arxiv import load_dataset, get_symmetric_graph
from src.train.node_classifier import WeigthedGraphSageNodeClassifier

In [3]:
logging.basicConfig(
    format='%(asctime)s - %(levelname)s : %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S'
)

In [4]:
torch.cuda.is_available()

True

In [5]:
device = f'cuda:0' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)
device

device(type='cuda', index=0)

## Data Loading

In [6]:
dataset = load_dataset()

In [7]:
graph = get_symmetric_graph(dataset)

In [8]:
graph.number_of_edges()

2332486

In [9]:
torch.mean(graph.out_degrees() * 1.)

tensor(13.7737)

In [10]:
torch.mean(graph.in_degrees() * 1.)

tensor(13.7737)

In [11]:
split_idx = dataset.get_idx_split()

In [12]:
split_idx

{'train': tensor([     0,      1,      2,  ..., 169145, 169148, 169251]),
 'valid': tensor([   349,    357,    366,  ..., 169185, 169261, 169296]),
 'test': tensor([   346,    398,    451,  ..., 169340, 169341, 169342])}

In [13]:
edges_gamma_df = pd.read_csv('data/gamma_graph_sage/01-1_gamma_scored_edges.csv')
print(edges_gamma_df.shape)
edges_gamma_df.head()

(2315598, 3)


,source,target,gamma_pred
0,411,0,0.649669
1,640,0,0.784250
2,1162,0,0.787807
3,1897,0,0.787201
4,3396,0,0.784839


In [14]:
keys = tuple(zip(edges_gamma_df['source'].values, edges_gamma_df['target'].values))
pred_dict = dict(zip(keys, edges_gamma_df['gamma_pred'].values))

In [15]:
idx=0
pred_dict[(graph.edges()[0][idx].item(), graph.edges()[1][idx].item())]

0.74366045

In [16]:
source = graph.edges()[0].numpy()
target = graph.edges()[1].numpy()
graph_edge_keys = tuple(zip(source, target))

In [17]:
base_weights = [pred_dict[edge] for edge in graph_edge_keys]
base_weights = torch.tensor(base_weights, dtype=torch.float32).view(-1, 1).to(device)
base_weights[:5]

tensor([[0.7437],
        [0.7242],
        [0.5969],
        [0.7571],
        [0.7405]], device='cuda:0')

In [18]:
weigher = EdgeWeigher(base_weights).to(device)
weigher.reset_parameters()

In [19]:
features = graph.ndata['feat'].cuda()
labels = dataset.labels.cuda()
train_mask = split_idx['train'].cuda()
val_mask = split_idx['valid'].cuda()
test_mask = split_idx['test'].cuda()

In [20]:
train_nid = train_mask.nonzero().squeeze()
val_nid = val_mask.nonzero().squeeze()
test_nid = test_mask.nonzero().squeeze()

In [21]:
n_edges = graph.number_of_edges()

In [22]:
graph

Graph(num_nodes=169343, num_edges=2332486,
      ndata_schemes={'year': Scheme(shape=(1,), dtype=torch.int64), 'feat': Scheme(shape=(128,), dtype=torch.float32)}
      edata_schemes={})

In [23]:
torch.version.cuda

'11.3'

In [24]:
graph = graph.to(device)

In [25]:
n_layers = 3
n_iters = 3000
epochs = 5000
log_steps = 100
input_dim = features.shape[1]
hidden_channels = input_dim * 2
output_dim = dataset.num_classes
lr_rate = 0.001
dropout = 0.5

In [26]:
model = GraphSAGE(
    n_layers=n_layers,
    in_channels=input_dim,
    hidden_channels=hidden_channels,
    out_channels=output_dim,
    dropout=dropout)\
    .to(device)

In [27]:
def train(model, weigher, graph, features, train_mask, optimizer):
    model.train()
    weigher.train()

    edge_weight = weigher()
    optimizer.zero_grad()
    out = model(graph, features, edge_weight=edge_weight)[train_mask]
    loss = F.nll_loss(out, labels.squeeze(1)[train_mask])
    loss.backward()
    optimizer.step()

    return loss.item()

In [28]:
@torch.no_grad()
def test(model, weigher, graph, features, labels, train_mask, val_mask, test_mask, evaluator):
    model.eval()
    weigher.eval()

    edge_weight = weigher()
    out = model(graph, features, edge_weight=edge_weight)
    y_pred = out.argmax(dim=-1, keepdim=True)

    train_acc = evaluator.eval({
        'y_true': labels[train_mask],
        'y_pred': y_pred[train_mask],
    })['acc']
    valid_acc = evaluator.eval({
        'y_true': labels[val_mask],
        'y_pred': y_pred[val_mask],
    })['acc']
    test_acc = evaluator.eval({
        'y_true': labels[test_mask],
        'y_pred': y_pred[test_mask],
    })['acc']

    return train_acc, valid_acc, test_acc

In [29]:
evaluator = Evaluator(name='ogbn-arxiv')

In [30]:
optimizer = torch.optim.Adam(
    list(model.parameters()) +
    list(weigher.parameters()),
    lr=lr_rate)
for epoch in range(1, 1 + epochs):
    loss = train(model, weigher, graph, features, train_mask, optimizer)
    result = test(model, weigher, graph, features, labels, train_mask, val_mask, test_mask, evaluator)

    if epoch % log_steps == 0:
        train_acc, valid_acc, test_acc = result
        print(f'Epoch: {epoch:02d}, '
              f'Loss: {loss:.4f}, '
              f'Train: {100 * train_acc:.2f}%, '
              f'Valid: {100 * valid_acc:.2f}% '
              f'Test: {100 * test_acc:.2f}%')

Epoch: 100, Loss: 1.3725, Train: 64.98%, Valid: 65.68% Test: 64.79%
Epoch: 200, Loss: 1.1446, Train: 69.44%, Valid: 68.98% Test: 67.95%
Epoch: 300, Loss: 1.0526, Train: 71.22%, Valid: 70.17% Test: 69.06%
Epoch: 400, Loss: 1.0034, Train: 72.47%, Valid: 70.78% Test: 69.77%
Epoch: 500, Loss: 0.9638, Train: 73.35%, Valid: 71.08% Test: 70.03%
Epoch: 600, Loss: 0.9308, Train: 74.14%, Valid: 71.52% Test: 70.36%
Epoch: 700, Loss: 0.9062, Train: 74.80%, Valid: 71.75% Test: 70.78%
Epoch: 800, Loss: 0.8843, Train: 75.35%, Valid: 71.79% Test: 70.46%
Epoch: 900, Loss: 0.8640, Train: 75.87%, Valid: 72.07% Test: 70.90%
Epoch: 1000, Loss: 0.8426, Train: 76.40%, Valid: 72.17% Test: 70.98%
Epoch: 1100, Loss: 0.8298, Train: 76.89%, Valid: 72.24% Test: 71.14%
Epoch: 1200, Loss: 0.8157, Train: 77.33%, Valid: 72.30% Test: 71.23%
Epoch: 1300, Loss: 0.7998, Train: 77.81%, Valid: 72.42% Test: 71.35%
Epoch: 1400, Loss: 0.7891, Train: 78.22%, Valid: 72.50% Test: 71.38%
Epoch: 1500, Loss: 0.7771, Train: 78.60%, V

In [31]:
weigher.lin.weight

Parameter containing:
tensor([[2.6578]], device='cuda:0', requires_grad=True)

In [32]:
weigher.lin.bias

Parameter containing:
tensor([1.6175], device='cuda:0', requires_grad=True)